# Pydap

[Pydap](http://pydap.org/) [[github](https://github.com/pydap/pydap)]

## Installing PyDap

Pydap is on conda-forge, however the handlers and responses are only on PyPI, as well as server.

It seems necessary to install [Handlers](http://pydap.org/en/latest/handlers.html), e.g.

    pip install Pydap[handlers.netcdf]

Similarly there are [Responses](http://pydap.org/en/latest/responses.html) that may be installed, e.g.

    pip install pydap.responses.netcdf

Installing the netcdf response allows `.nc` to be appended.


In [ ]:
import os
from pprint import pprint

import numpy as np
import rasterio as rio
from rasterio.crs import CRS
import xarray as xr

## OPeNDAP Review

- [QuickStart - OPeNDAP Documentation](https://opendap.github.io/documentation/QuickStart.html)
- [User Guide - OPeNDAP Documentation](https://opendap.github.io/documentation/UserGuideComprehensive.html)
- [The Hyrax Data Server Installation and Configuration Guide - OPeNDAP Documentation](https://opendap.github.io/hyrax_guide/Master_Hyrax_Guide.html)

Suffixes:

- `.info`---may be most useful!
- `.html`
- `.dds`
- `.das`
- `.dods`
- `.ascii`
- ...
- `.rdf`?

- `.ddx`?

DAP 4:

- `dmr`/`dmr.html`?/`dmr.xml`/`dmr.txt`?---`.dmr`~`.dds`+`das` from DAP2
- `dap`---~`.dods`
- **`dsr`**---returns dataset services, new in DAP4
- ...
- 

Returns (downloads?) in native OPeNDAP binary ecoding (`.dap`), NetCDF (`.nc`, `.nc4`), GeoTIFF (`.tiff`), JPEG2000, JSON (`.json`), and ASCII (`.ascii`). ???

Misc:

[Authentication For DAP Clients](https://opendap.github.io/hyrax_guide/Master_Hyrax_Guide.html#_authentication_for_dap_clients) discusses Earthdata, etc. An apparently older verion is at [DAP Clients - Authenticaion](https://docs.opendap.org/index.php/DAP_Clients_-_Authentication).

https://docs.opendap.org/index.php?title=DAP4_Specification

- [DAP4 Specification Volume 2: DAP4 Web Services](https://docs.opendap.org/index.php?title=DAP4:_Specification_Volume_2) is useful, incl. [8 Appendix - Ancillary Web Services (Beyond DAP4)](https://docs.opendap.org/index.php?title=DAP4:_Specification_Volume_2#Appendix_-_Ancillary_Web_Services_.28Beyond_DAP4.29)

https://docs.opendap.org/index.php?title=OPULS_Development

https://docs.opendap.org/index.php?title=DAP4_Dataset_Services_Response

https://www.unidata.ucar.edu/software/thredds/v4.6/tds/tutorial/DAP.html --- DAP 4 has new/different suffixes, not comprehensive!

[Aggregation enhancements ](https://docs.opendap.org/index.php/Aggregation_enhancements) discusses [NcML](https://www.unidata.ucar.edu/software/thredds/current/netcdf-java/ncml/).

The relevance of the file extension on a DAP server is unclear!? 


## Pydap Classes

Pydap implemements various classess to represent [The DAP data model](http://pydap.org/en/latest/developer.html#the-dap-data-model). The [Developer documentation](http://pydap.org/en/latest/developer.html) also sheds some light on things.

PyDap Types:

- `DatasetType`---a `StructureType`
- `BaseType`---data is scalar or ndarray
- `GridType`---strangely stored with 3 vhildren, the first is the array (BaseType) and the other two are axes
- `SequenceType`---includes `iterdata()` method
- `StructureType`---also behave like Python dict

PyDap types have attributes:

1. `name`
2. `id`
3. `attributes`---allow lazy access (with dot)
4. `data`

Some (which?) have:

- `dtype`
- `dimensions`
- ...

You'll generally get data through a variable's `data` attribute, but note that it may be necessary to slice it like `dataset.variable[:].data` for dap to handle it properly.

StructureTypes hold no data, but their `data` attribute collects data recursively from its children. Most Dap types seem to have a `children()` method but only seems relevant to StructureTypes. At least GridTypes have an `array` attribute for accessing BaseType array, so you may have to something like `dataset.variable.array[:].data`. 

Note that a dataset may contain a variable containing no actual data but with useful attributes!

Note that variables usually act like numpy arrays, incl. things like `shape()` method.


In [ ]:
from pydap.model import DatasetType, SequenceType, GridType, StructureType, BaseType, DapType
assert issubclass(BaseType, DapType)
assert issubclass(StructureType, DapType)
assert issubclass(DatasetType, StructureType)
assert issubclass(SequenceType, StructureType)
assert issubclass(GridType, StructureType)

## Using Pydap

[Using the client](http://pydap.org/en/latest/client.html)

Create `DatasetType` which is a fancy dict structure.

Get variables using dictionary syntax or attribute style access.

Variables can be `GridType`, ...?


In [ ]:
from pprint import pprint

In [ ]:
# A basic import is useless, it only gets you handlers, responses, and tests as empty namespaces!
# To do anything you probably need to import directly. Importing these namespaces will enable tab completion for their contents.
import pydap

In [ ]:
# Not sure this works, it should have to be installed separately.
# import pydap.handlers.netcdf

In [ ]:
# Pydap is bizarre, 
help(pydap)
help(pydap.handlers)
help(pydap.responses)
# help(pydap.tests)

In [ ]:
from pydap.client import open_url
url = 'http://www.ncei.noaa.gov/thredds/dodsC/namanl/201604/20160416/namanl_218_20160416_1800_000.grb'
dataset = open_url(url)  # Create DatasetType.

In [ ]:
pprint(dataset.name)
pprint(dataset.id)
pprint(dataset.attributes)
# pprint(dataset.data)  # Long output.

# list(dataset.keys())

# For fun, get types of the variables in the dataset.
# pprint([type(dataset[k]) for k, v in dataset.items()])

In [ ]:
dataset["Temperature_surface.Temperature_surface"]
dataset["Temperature_surface"]["Temperature_surface"]

### NASA Earthdata with Pydap

Pydap used to support Earthdata auth using `pydap.utils.urs.install_basic_client()` (which seemed to read `.netrc`), but `pydap.utils` (which had to be separately installed) has gone away (no later than 1/2017). Now Earthdata auth is done using `pydap.cas.urs.setup_session()` to obtain a `requests.sessions.Session` object which is then passed to `open_url`.


The info (`.html`) page allows you to download as NetCDF 3 or 4, or as DAP 2 or 4.

The OPeNDAP directory listing ddx, dds, das, info, html, rdf, covjson, plus viewers:


Notes:

- The Dataset Viewers (viewers) leads you to:
  - DAP2
  - DAP4
  - w10n Service (Webification)---adds slash after file name!


Auth:

- [How To Access Data With PyDAP | Earthdata Wiki](https://wiki.earthdata.nasa.gov/display/EL/How+To+Access+Data+With+PyDAP) discusses using `pydap.util.urs` module but that's out of date! Other examples may work?
- [How To Access Data With Python](https://wiki.earthdata.nasa.gov/display/EL/How+To+Access+Data+With+Python) might work?


## SRTM Using OPeNDAP

NASA SRTM Version 3 is available through [OPeNDAP](https://lpdaac.usgs.gov/tools/opendap/) as 
[`.ncml` files](https://opendap.cr.usgs.gov/opendap/hyrax/SRTMGL1.003/contents.html). I can't figure out how to use them as `ncml` but thankfully there is a `netcdf` directory containing [`.nc` files](https://opendap.cr.usgs.gov/opendap/hyrax/SRTMGL1.003/netcdf/contents.html).

While navigating, the following services are presented for SRTM netcdf data:

- [ddx](https://opendap.cr.usgs.gov/opendap/hyrax/SRTMGL1.003/netcdf/N37W120.SRTMGL1.nc.ddx)---xml
- [dds](https://opendap.cr.usgs.gov/opendap/hyrax/SRTMGL1.003/netcdf/N37W120.SRTMGL1.nc.dds)
- [das](https://opendap.cr.usgs.gov/opendap/hyrax/SRTMGL1.003/netcdf/N37W120.SRTMGL1.nc.das)
- [info](https://opendap.cr.usgs.gov/opendap/hyrax/SRTMGL1.003/netcdf/N37W120.SRTMGL1.nc.info)---nice overview
- [html](https://opendap.cr.usgs.gov/opendap/hyrax/SRTMGL1.003/netcdf/N37W120.SRTMGL1.nc.html)---same as DAP2 Service from viewers page, and file link
- [rdf](https://opendap.cr.usgs.gov/opendap/hyrax/SRTMGL1.003/netcdf/N37W120.SRTMGL1.nc.rdf)
- [covjson](https://opendap.cr.usgs.gov/opendap/hyrax/SRTMGL1.003/netcdf/N37W120.SRTMGL1.nc.covjson)

There is also a link to viewers, which include links to the DAP2 Service (same as `html` link above) and DAP4 Service:

- [dmr.html (DAP4 Service)](https://opendap.cr.usgs.gov/opendap/hyrax/SRTMGL1.003/netcdf/N37W120.SRTMGL1.nc.dmr.html)

Other useful services:

- [dsr](https://opendap.cr.usgs.gov/opendap/hyrax/SRTMGL1.003/netcdf/N37W120.SRTMGL1.nc.dsr)---lists all responses!

The GDAL documentation for the [NetCDF: Network Common Data Form](https://www.gdal.org/frmt_netcdf.html) may be useful in deciphering the dataset.


Relevant issues:

- https://github.com/pydap/pydap/issues/20 ---!
- https://github.com/pydap/pydap/issues/19
- https://github.com/pydap/pydap/pull/28 ---PR merged
- https://github.com/pydap/pydap/pull/11
- https://github.com/pydap/pydap/pull/26
- https://github.com/pydap/pydap/issues/51

### SRTM Aside

`.hgt` in 16-bit binary signed interger, in meter, row-major order, big-endian (note Intel is little-endian).

Names refer to SW corner, geometric center of lower-left pixel.


### Open as Pydap DatasetType

In [ ]:
# Read auth information from .netrc.
import netrc
auth = netrc.netrc()
login, account, password = auth.authenticators('urs.earthdata.nasa.gov')
# login, account, password

In [ ]:
from pydap.client import open_url        # Or import pydap.client to make available.
from pydap.cas.urs import setup_session  # Or import pydap.cas.urs to make available.

# dataset_url = 'https://opendap.cr.usgs.gov/opendap/hyrax/SRTMGL1.003/N37W120.SRTMGL1.ncml'
# Can't figure out how to work with nclm files, but thankfully SRTM is also available as netcdf.
dataset_url = 'https://opendap.cr.usgs.gov/opendap/hyrax/SRTMGL1.003/netcdf/N37W120.SRTMGL1.nc'

# Use login/password read from .netrc above.
session = setup_session(username=login, password=password, check_url=dataset_url)
dataset = open_url(dataset_url, session=session)
dataset

Pydap objects contain the key attributes `name`, `id`, `attributes`, and `data`.

In [ ]:
print(dataset.name)
print(dataset.id)

In [ ]:
dataset.attributes

In [ ]:
dataset.data

While `data` in DatasetType recursively contains the data of all its children we can also list the immediate children. 

In [ ]:
list(dataset.keys())

In [ ]:
list(dataset.children())
# list(dataset.items())

In [ ]:
[(k, v.attributes) for k, v in dataset.items()]

In [ ]:
# For this SRTM dataset the lat and lon variables seem redundant.
assert np.array_equal(dataset.lat[:].data, dataset.Band1.lat[:].data)
assert np.array_equal(dataset.lon[:].data, dataset.Band1.lon[:].data)

In [ ]:
# Here are a few other urls that use Earthdata login.

# ASTER DEM, works!
# dataset_url = 'https://opendap.cr.usgs.gov/opendap/hyrax/ASTGTM.002/ASTGTM2_N37E120_dem.nc'

# Another server, works!
# dataset_url = 'https://measures.gesdisc.eosdis.nasa.gov:443/opendap/LANDMET/LANDMET_ANC_SM.1/LANDMET_ANC_SM_L3_v1.nc'

In [ ]:
# # Explore...
# pydap.cas.urs.setup_session??

# # Explore...
# pydap.cas.urs.get_cookies.setup_session??

### Open as xarray Dataset using Pydap

The ability to use authentication to open datasets in xarray using Pydap was added in PR [xarray#1570](https://github.com/pydata/xarray/pull/1570).

Interaction is similar to using Pydap directly. Pydap `attributes` are available in xarray `attr`. However it seems that the explicit call to `.data` is not necessary as it is with Pydap, which also avoids the weirdness of GridTypes.

Here we give an example using SRTM data.


In [ ]:
# Open authenticated datasets with xarray using pydap backend.
# Pattern from https://github.com/pydata/xarray/pull/1570.

import xarray as xr
import pydap.cas.urs  # Make setup_session() available.
import pydap.client   # Make open_url() available.

dataset_url = 'https://opendap.cr.usgs.gov/opendap/hyrax/SRTMGL1.003/netcdf/N37W120.SRTMGL1.nc'

# Instantiate session.
session = pydap.cas.urs.setup_session(username=login, password=password, check_url=dataset_url)

# Method 1: More verbose.
# pydap_ds = pydap.client.open_url(dataset_url, session=session)
# store = xr.backends.PydapDataStore(pydap_ds)
# Method 2: More concise but equivalent.
store = xr.backends.PydapDataStore.open(dataset_url, session=session)

# Create xarray Dataset.
ds = xr.open_dataset(store)
ds

Dataset:

- dims, data_vars, coords, attrs
- ...

In [ ]:
# Get types of Dataset key properties.
print(type(ds.dims))
print(type(ds.data_vars))
print(type(ds.coords))
print(type(ds.attrs))

In [ ]:
ds.dims

In [ ]:
ds.data_vars

In [ ]:
ds.coords

In [ ]:
ds.attrs

---

### ds?

In [ ]:
# 
ds.crs.attrs

In [ ]:
crs = ds.crs

# crs.values
pprint(crs.dims)
pprint(crs.coords)
pprint(crs.attrs)
pprint(crs.name)

---

## Compare to Local DEM

The original HGT files may contain more metadata than the versions accessed through DAP.


In [ ]:
from affine import Affine

In [ ]:
lon = 'W120'
lat = 'N37'
url = 'https://e4ftl01.cr.usgs.gov/MEASURES/SRTMGL1.003/2000.02.11/' + lat + lon + '.SRTMGL1.hgt.zip'
url

Download an example SRTMGL1 file.

In [ ]:
# Download SRTMGL1 tile to cwd.
if not os.path.isfile(os.path.basename(url)):
    # Based on https://wiki.earthdata.nasa.gov/display/EL/How+To+Access+Data+With+cURL+And+Wget.
    !touch ~/.urs_cookies
    !curl -O -b ~/.urs_cookies -c ~/.urs_cookies -L -n '{url}'
    # !unzip '{os.path.basename(url)}'

This script may be useful for downloading multiple files.

In [ ]:
# %%bash
# # Based on https://wiki.earthdata.nasa.gov/display/EL/How+To+Access+Data+With+cURL+And+Wget.

# touch ~/.urs_cookies

# fetch_urls() {
#     while read -r line; do
#     curl -b ~/.urs_cookies -c ~/.urs_cookies -L -n -f -Og $line && echo || exit_with_error "Command failed with error. Please retrieve the data manually."
#     done;
# }
# fetch_urls <<'EDSCEOF'
# https://e4ftl01.cr.usgs.gov/MEASURES/SRTMGL1.003/2000.02.11/N37W120.SRTMGL1.hgt.zip
# EDSCEOF

# # Explanation of curl options:
# # -O
# # -b
# # -c
# # -L
# # -n

### Open as Rasterio dataset

Rasterio can open SRTM files using the `SRTMHGT` GDAL driver, either as `*.hgt` or directly from the zip.

In [ ]:
# Open the *.hgt.zip file with Rasterio.
filename = os.path.abspath(os.path.basename(url))
dataset = rio.open(filename, driver='SRTMHGT')  # The driver is optional.
# pprint(dataset.meta)
pprint(dataset.profile)

In [ ]:
# Compare dataset attributes of the Rasterio object with the output of gdalinfo.
! gdalinfo '{filename}'
# ! gdalinfo -proj4 '{filename}'
# ! gdalinfo -json '{filename}'
# ! gdalinfo -listmdd '{filename}'

In [ ]:
# Note GEOGCS implies a WKT representation.
pprint(dataset.crs.to_dict())
pprint(dataset.crs.to_epsg())
pprint(dataset.crs.to_proj4())
pprint(dataset.crs.to_string())
pprint(dataset.crs.to_wkt())

In [ ]:
rio.crs.CRS.from_wkt(dataset.crs.to_wkt())

In [ ]:
dataset.transform

In [ ]:
dataset.get_transform()

In [ ]:
print(dataset.driver)
print(dataset.files)
print(dataset.shape)  # ~ width and height attributes.

In [ ]:
print(dataset.res)
print(dataset.tags())

In [ ]:
print(dataset.bounds)
print(dataset.lnglat())

In [ ]:
print(dataset.block_shapes)
print(dataset.dtypes)
print(dataset.colorinterp)
print(dataset.nodatavals)  # nodata just returns the first element on nodatavals
print(dataset.units)

### Open as xarray DataArray

In [ ]:
# Open SRTM file with xarray.
da = xr.open_rasterio(dataset)
assert da.identical(xr.open_rasterio(filename))  # Could also open directly from filename.
da

DataArray:

- values, dims, coords, attrs
- name
- data

In [ ]:
# Get types of DataArray key properties.
print(type(da.values))
print(type(da.dims))
print(type(da.coords))
print(type(da.attrs))

In [ ]:
da.values

In [ ]:
da.dims

In [ ]:
da.coords

In [ ]:
da.attrs

Attributes for the xarray DataArray are populated from some of the attributes of the Rasterio dataset.

Remember that xr attributes may be read with attribute style access but live in `attrs`. 

Note the meta and profile attributes of Rasterio dataset also contain much of this information.

It doesn't see that the Rasterio `units` attribute is reflected in xarray!

In [ ]:
# In Rasterio the transform attribute is an Affine instance.
assert np.array_equal(da.transform, dataset.transform[0:6])
assert dataset.transform.almost_equals(da.transform)
assert rio.transform.Affine(*da.transform) == dataset.transform

assert da.crs == dataset.crs.to_string()
assert da.res == dataset.res

# In Rasterio is_tiled is a bool, but it's stored as a uint8 attribute in xarray.
assert da.is_tiled == dataset.is_tiled

assert da.nodatavals == dataset.nodatavals

In [ ]:
da.dtype == dataset.dtypes[0]

In [ ]:
Affine.from_gdal(*da.transform)

#### Clean DataArray

In [ ]:
# Clean up DataArray that was opened from HGT file.

# da_clean = da.copy()
# da_clean.name = 'elevation'
da_clean = da.rename('elevation')  # Add name property.
da_clean = da_clean.squeeze(drop=True)    # Remove length 1 dimensions and their coordinates, in particular band.
da_clean

In [ ]:
# This is what the DataArray would look like converted to a Dataset.
da_clean.to_dataset()

#### Clean Dataset?

In [ ]:
ds

In [ ]:
ds.info()

In [ ]:
ds.Band1

In [ ]:
ds.crs

In [ ]:
ds.lat

In [ ]:
ds.lon

In [ ]:
# Clean up Dataset that was opened from DAP, as DataSet.

ds_clean = ds.rename({'Band1': 'elevation'})  # Rename Band1 variable to elevation.
# ds_clean = ds_clean.squeeze(drop=True)    # Remove length 1 dimensions and their coordinates, in particular band.
ds_clean

# Add crs attribute.
wkt = ds.crs.spatial_ref.replace('\\"', '"')
crs = CRS.from_wkt(wkt).to_string()
# Add crs attribute to elevation variable and update Dataset with it.
new_elevation =  ds_clean.elevation.assign_attrs(crs = crs)  # Seems same as assign_attrs({'crs' : crs}).
ds_clean = ds_clean.assign(elevation = new_elevation)  # Works.

# Add transform attribute.
geotransform = [float(s) for s in ds.crs.GeoTransform.split()]
affine = rio.transform.Affine.from_gdal(*geotransform)
new_elevation =  ds_clean.elevation.assign_attrs(transform = tuple(affine)[0:6])
ds_clean = ds_clean.assign(elevation = new_elevation)

# TODO change to make a crs attribute with the stuff in there?
# Add all attrs from strange crs variable to elevation variable and update Dataset with this new value.
new_elevation = ds_clean.elevation.assign_attrs(ds_clean.crs.attrs)
ds_clean = ds_clean.assign(elevation = new_elevation)  # Works.
# ds_clean = ds_clean.assign({'elevation': new_elevation})  # Works.
# ds_clean = ds_clean.update({'elevation': new_elevation})  # Works.
# ds_clean['elevation'] = new_elevation  # Works.

# TODO remove extra junk attrs copied from crs variable to elevation variable.

# Remove unneeded crs variable.
ds_clean = ds_clean.drop('crs')

ds_clean.elevation

In [ ]:
# Make sure that transforms are comparable.
assert Affine(*ds_clean.elevation.transform).almost_equals(da_clean.transform)
assert np.allclose(ds_clean.elevation.transform, da_clean.transform)

---

In [ ]:
elevation = ds_clean.elevation

In [ ]:
elevation.interp_like?

In [ ]:
# da.where(da.x<-119.5)

In [ ]:
# da.sel(x=slice(-120, -119.5))

---

---

## Work with CRS

Make sure to:

    from rasterio.crs import CRS


It appears that there is some weirdness wrt how `rio.crs.CRS` works. The `AUTHORITY` parts of `PRIMEM` and `UNIT` don't seem to make it to Dataset `crs.spatial_ref` when opening the files locally. The CRS object created from this has the correct EPSG but the `wkt` attribute is still missing these. However, creating a new CRS object using that EPSG string results in the complete WKT.

The `crs.spatial_ref` string in the xarray Dataset obtained through DAP is a funny thing. It seems to contain the pattern `\\"`, which seems to be a

In the NetCDF dataset the crs.spatial_ref contains escaped double quotes (`\"`). However when accessing with Pydap or xarray they are converted into `\\"` for some reason. I believe the second backslash is added in an attempt to escape the string, ignoring the fact that it's already escaped. The resulting string is malformed and can't be fed directly to rasterio.crs.CRS functions.

### WKT

https://www.opengeospatial.org/standards/wkt-crs


In [ ]:
# Illustrate that escapes are optional.
"\"" == '"'

In [ ]:
ds.crs

In [ ]:
# Rasterio crs.
dataset.crs.wkt

In [ ]:
# DataArray crs attribute comes from Rasterio.
da.crs

In [ ]:
# Convert string back to WKT.
CRS.from_string(da.crs).wkt

In [ ]:
# print() just shows the backslash.
print(ds.crs.spatial_ref)

In [ ]:
# str() shows the 
str(ds.crs.spatial_ref)

In [ ]:
ds.crs.spatial_ref.__str__()

In [ ]:
ds.crs.spatial_ref

In [ ]:
# Note that repr() even adds escaped opening and closing single quotes.
repr(ds.crs.spatial_ref)

In [ ]:
ds.crs.spatial_ref.__repr__()

In [ ]:
wkt = ds.crs.spatial_ref

#???
# This seems to work on the string shown by str().
# Note print() and str() use __str__(), but escapes prob handled differently, while repr() uses __repr__().



wkt = ds.crs.spatial_ref.replace('\\"', '"') # Escape just the backslash.
# wkt = ds.crs.spatial_ref.replace("\\\"", "\"")  # Escape both backslash and quote characters.
# wkt = ds.crs.spatial_ref.replace('\\', '')  # Just replace the backslash, but better to include the quote as above.
# print(wkt)

crs = CRS.from_wkt(wkt)
# CRS.from_string(wkt)
# CRS.from_user_input(wkt)
print(crs)

# wkt
print(wkt)
# str(wkt)
# repr(wkt)

In [ ]:
# # TODO maybe later, do the same with re.sub().
# import re
# re.sub('\\', '', ds.crs.spatial_ref)